In [2]:
"""
    CNN example #2_day4 (2017.09.17)

"""

import tensorflow as tf
sess = tf.InteractiveSession()

#
# Image data load 
#

# Training images


import numpy as np
# import matplotlib.pyplot as plt

fd = open('./Desktop/mnist/train-images.idx3-ubyte')
images = np.fromfile(file=fd, dtype=np.uint8)
images = images[16:].reshape([60000, 28, 28]).astype(np.float)
images = images/127.0-1.0

fd = open('./Desktop/mnist/train-labels.idx1-ubyte')
labels = np.fromfile(file=fd, dtype=np.uint8)
labels = labels[8:].reshape([60000]).astype(np.int)

# Test images

fd = open('./Desktop/mnist/t10k-images.idx3-ubyte')
images_test = np.fromfile(file=fd, dtype=np.uint8)
images_test = images_test[16:].reshape([10000, 28, 28]).astype(np.float)
images_test = images_test/127.0-1.0

fd = open('./Desktop/mnist/t10k-labels.idx1-ubyte')
labels_test = np.fromfile(file=fd, dtype=np.uint8)
labels_test = labels_test[8:].reshape([10000]).astype(np.int)

print(images.shape)
print('----------End of image loading---------')

#
# Convolution
#

## First convolution layer
# weight, bias 변수 선언

conv1_weights = tf.Variable(tf.truncated_normal([5, 5, 1, 6], stddev=0.1, dtype=tf.float32))
conv1_biases = tf.Variable(tf.zeros([6], dtype=tf.float32))
print(conv1_weights)

# Conv layer 선언 (tf.nn.conv2d, tf.nn.bias_add)

X = tf.placeholder(tf.float32, [None, 28, 28, 1], name='X')                   
Y = tf.placeholder(shape=[None], dtype=tf.int64, name='Y')
is_training = tf.placeholder(tf.bool)

net = tf.nn.conv2d(X, conv1_weights, [1, 1, 1, 1], padding='SAME', name='conv1')
net = tf.nn.bias_add(net, conv1_biases)

# Activation functions 

net = tf.nn.relu(net)

#  Pooling layer (tf.nn.max_pool)
net = tf.nn.max_pool(net, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME', name='pool1')


## Second convolution layer
# weight, bias 변수 선언

conv2_weights = tf.Variable(tf.truncated_normal([5, 5, 6, 16], stddev=0.1, dtype=tf.float32))
conv2_biases = tf.Variable(tf.zeros([16], dtype=tf.float32))


# Conv layer 선언 (tf.nn.conv2d, tf.nn.bias_add)

net = tf.nn.conv2d(net, conv2_weights, [1, 1, 1, 1], padding='VALID', name='conv2')
net = tf.nn.bias_add(net, conv2_biases)

# Activation functions 

net = tf.nn.relu(net)

#  Pooling layer (tf.nn.max_pool)
net = tf.nn.max_pool(net, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME', name='pool1')

## Third convolution layer
# weight, bias 변수 선언

conv3_weights = tf.Variable(tf.truncated_normal([5, 5, 16, 120], stddev=0.1, dtype=tf.float32))
conv3_biases = tf.Variable(tf.zeros([120], dtype=tf.float32))


# Conv layer 선언 (tf.nn.conv2d, tf.nn.bias_add)

net = tf.nn.conv2d(net, conv3_weights, [1, 1, 1, 1], padding='VALID', name='conv3')
net = tf.nn.bias_add(net, conv3_biases)

# Activation functions 

net = tf.nn.relu(net)

print(net)
print('----------End of CONVOLUTION---------')

#
# Neural network
#

# Flatten
net = tf.reshape(net,[-1,120])
print(net)

# variables
fc1_weights = tf.Variable(tf.truncated_normal([120, 84], stddev=0.1, dtype=tf.float32))
fc1_biases = tf.Variable(tf.constant(0.1, shape=[84], dtype=tf.float32))

# Linear function
net = tf.nn.tanh(tf.matmul(net, fc1_weights) + fc1_biases)

# variables
fc2_weights = tf.Variable(tf.truncated_normal([84, 10], stddev=0.1, dtype=tf.float32))
fc2_biases = tf.Variable(tf.ones([10], dtype=tf.float32))


# Output
Output = tf.nn.tanh(tf.matmul(net, fc2_weights) + fc2_biases)
                
# loss function
Y_onehot = tf.one_hot(Y, 10, axis=1)
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=Output, labels=Y_onehot))

# Optimizer
trainer = tf.train.GradientDescentOptimizer(learning_rate=0.05)
optimize = trainer.minimize(loss)

# Softmax
softmax = tf.nn.softmax(Output)

# Accuracy
correct = tf.equal(tf.argmax(softmax, 1), tf.argmax(Y_onehot, 1))
acc = tf.reduce_mean(tf.cast(correct, tf.float32))

#
# Training
#

# tensorflow variable init
sess.run(tf.global_variables_initializer())
    
batch_size = 100
batch_count = 60000 // batch_size
acc_train_list= []
    
for epoch in range(100):
    # epoch loop
    total_loss = 0
        
    for i in range(batch_count):
        # mini-batch loop
        batch_index = i * batch_size
        img = np.reshape(images[batch_index:batch_index+batch_size], [batch_size, 28, 28, 1])
        label = labels[batch_index:batch_index+batch_size]
        _, loss_v = sess.run([optimize, loss], feed_dict={X:img, Y:label, is_training:True})
        total_loss += loss_v
        
    print('Epoch:','%04d' % (epoch + 1), 'total_loss:', '%04f' % (total_loss))
    Accuracy_training = sess.run(acc, feed_dict={X: img, Y: label, is_training: False})
    acc_train_list.append(Accuracy_training)
              

Accuracy = sess.run(acc, feed_dict={X: np.reshape(images_test,[10000, 28, 28, 1]), Y: labels_test, is_training: False})
print('Accuracy:','%04f' %(Accuracy*100))
# print(acc_train_list)

(60000, 28, 28)
----------End of image loading---------
<tf.Variable 'Variable_10:0' shape=(5, 5, 1, 6) dtype=float32_ref>
Tensor("Relu_5:0", shape=(?, 1, 1, 120), dtype=float32)
----------End of CONVOLUTION---------
Tensor("Reshape_4:0", shape=(?, 120), dtype=float32)
('Epoch:', '0001', 'total_loss:', '698.742597')
('Epoch:', '0002', 'total_loss:', '536.169706')
('Epoch:', '0003', 'total_loss:', '518.747885')
('Epoch:', '0004', 'total_loss:', '510.690798')
('Epoch:', '0005', 'total_loss:', '505.687921')
('Epoch:', '0006', 'total_loss:', '502.194388')
('Epoch:', '0007', 'total_loss:', '499.597013')
('Epoch:', '0008', 'total_loss:', '497.614932')
('Epoch:', '0009', 'total_loss:', '495.997274')
('Epoch:', '0010', 'total_loss:', '494.647779')
('Epoch:', '0011', 'total_loss:', '493.469868')
('Epoch:', '0012', 'total_loss:', '492.436524')
('Epoch:', '0013', 'total_loss:', '491.530665')
('Epoch:', '0014', 'total_loss:', '490.743371')
('Epoch:', '0015', 'total_loss:', '490.078069')
('Epoch:',